# INFO2950 Final Project Phase 2


## Research Question

Question: Can we reliably predict a New York county's wealth based on their percentage of postgraduates of a certain age and major?

At the conclusion of this project, we seek to determine if there is a correlation between college major and the per capita income of the postgraduate county of residence in New York, and age and the per capita income of the postgraduate county of residence in New York. We will train a multivariable regression on data tables from both Wikepedia and the U.S. Census Bureau to see if we can reliably predict a county's per capita income given the population percentages of Science and Engineering, Business, Education, and Arts, Humanities, and Others for each of these age groups: 25-39 years, 40-64 years, and 65 years and over. 

We defined county wealth using per capita income of the county and we solely considered New York counties for our regression. We defined postgraduates as people holding a Bachelor's degree older than 25 years old. 

## Data Description

We have two data tables from the U.S. Census Bureau and Wikepedia. 

Our data scraped from Wikepedia has one row for each New York State county and post-web scraping, there are two columns, one for county name and the other for the county's per capita income. Before we web scraped, there were additional columns for Median household income, Median family income, Population, and Number of households. We omitted these columns because they did not serve a purpose in our regression. This dataset was created by Wikepedia to inform the public of the income per capita of each county in New York State. The page is located at the link https://en.wikipedia.org/wiki/List_of_New_York_locations_by_per_capita_income.

We downloaded data from the U.S. Census Bureau's page: https://data.census.gov/table/ACSST1Y2022.S1502?g=040XX00US36,36 using New York State (as a whole) and all New York Counties as filters for their data set, created in 2022. The first column houses the labels of groups of people by age and college major; the following columns pre- data cleaning have each county and the general state's estimate of people falling into each category, the margin of error, percent estimate, percent error, male estimate, male margin of error, male percent estimate, male percent margin of error, female estimate, female margin of error, female percent estimate, and female percent margin of error. Each row is a classification of person, through age range and college major.

## Data Cleaning and Collection

### Data cleaning: getting rid of unnecessary columns of U.S. Census Bureau data table

### Data collection: web scraping from Wikepedia's "List of New York locations by per capita":

We attempted to data scrape the Wikepedia website, but we were unsuccessful because the < tr > tags that contained the data we were trying to scrape did not have a class associated with them, and TAs we consulted recommended manually pasting the data into an Excel spreadsheet and downloading it as a .csv file instead. Below, you will find our initial web scraping process, up to where we realized there was no clear way to scrape without a class in the < tr > tag. 

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb

In [3]:
wikepedia_url = "https://en.wikipedia.org/wiki/List_of_New_York_locations_by_per_capita_income"

In [5]:
wikepedia_result = requests.get(wikepedia_url)

In [6]:
with open("county_wealth.html", "w") as writer:
    writer.write(wikepedia_result.text)

In [9]:
with open("county_wealth.html", "r") as reader:
    html_source = reader.read()

In [10]:
# Confidence check
html_source[:20]

'<!DOCTYPE html>\n<htm'

In [11]:
page = BeautifulSoup(html_source, "html.parser")

Below, you will find that there are no classes in the < tr > tags that we scraped.

In [35]:
wikepedia_county_income = page.find("table", {"class": "wikitable"})
wikepedia_county_income = wikepedia_county_income.find_next('tbody').find_all('tr')
print(wikepedia_county_income)

[<tr valign="bottom">
<th>Rank
</th>
<th>County
</th>
<th>Per capita<br/>income
</th>
<th>Median<br/>household<br/>income
</th>
<th>Median<br/>family<br/>income
</th>
<th>Population
</th>
<th>Number of<br/>households
</th></tr>, <tr>
<td>1
</td>
<td><a href="/wiki/Manhattan" title="Manhattan">New York County</a>
</td>
<td>$111,386
</td>
<td>$64,971
</td>
<td>$75,629
</td>
<td>1,585,873
</td>
<td>763,846
</td></tr>, <tr>
<td>2
</td>
<td><a href="/wiki/Westchester_County,_New_York" title="Westchester County, New York">Westchester</a>
</td>
<td>$73,159
</td>
<td>$79,619
</td>
<td>$100,863
</td>
<td>949,113
</td>
<td>347,232
</td></tr>, <tr>
<td>3
</td>
<td><a href="/wiki/Nassau_County,_New_York" title="Nassau County, New York">Nassau</a>
</td>
<td>$41,387
</td>
<td>$93,613
</td>
<td>$107,934
</td>
<td>1,339,532
</td>
<td>448,528
</td></tr>, <tr>
<td>4
</td>
<td><a href="/wiki/Putnam_County,_New_York" title="Putnam County, New York">Putnam</a>
</td>
<td>$37,915
</td>
<td>$89,218
</td>
<td>

In [37]:
county_wealth = pd.read_csv("county_wealth.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'county_wealth.csv'

## Data Limitations

- The Census data is from 2022, so it may be outdated since people move counties frequently
- The Census data may not be as comprehensive due to many people dying of COVID during the pandemic